In [ ]:
import numpy as np
import sympy as smp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter

Define all necessary variables:

* Eor convinence consider true lenght spring $l$=1
* Time $t$
* Mass of pendulums $m$
* Gravitational acceleration $g$
* Spring constants $k$

Then define the 4 free variables $\theta_1$, $r_1$, $\theta_2$, $r_2$

1. Make them functions of time
2. Define first derivatives
3. Define second derivatives

In [ ]:
t, m1, m2, g, k1, k2 = smp.symbols('t m_1 m_2 g k_1 k_2')

In [ ]:
t1, t2, r1, r2 = smp.symbols(r'\theta_1, \theta_2, r_1, r_2', cls=smp.Function)
# m1 = m2
# k1 = k2
# theta1
t1 = t1(t)
w1 = smp.diff(t1, t)
dw1dt = smp.diff(smp.diff(t1, t), t)

t2 = t2(t)
w2 = smp.diff(t2, t)
dw2dt = smp.diff(smp.diff(t2, t), t)

r1 = r1(t)
v1 = smp.diff(r1, t)
dv1dt = smp.diff(smp.diff(r1, t), t)

r2 = r2(t)
v2 = smp.diff(r2, t)
dv2dt = smp.diff(smp.diff(r2, t), t)


Define cartesian coordinates of each bob

* Bob 1: $(x_1, y_1)$
* Bob 2: $(x_2, y_2)$

Note these are functions of $\theta_1$, $r_1$, $\theta_2$, $r_2$

In [ ]:
x1, y1, x2, y2 = smp.symbols('x_1, y_1, x_2, y_2', cls = smp.Function)
x1= x1(t1, r1)
y1= y1(t1, r1)
x2= x2(t1, r1, t2, r2)
y2= y2(t1, r1, t2, r2)

In [ ]:
x1 = (1+r1)*smp.cos(t1)
y1 = -(1+r1)*smp.sin(t1)
x2 = (1+r1)*smp.cos(t1) + (1+r2)*smp.cos(t2)
y2 = -(1+r1)*smp.sin(t1)-(1+r2)*smp.sin(t2)

Define the Lagrangian

$$ L = T - V $$

where 

* T = $\frac{1}{2}m(\dot{x}_1^2 + \dot{y}_1^2) + \frac{1}{2}m(\dot{x}_2^2 + \dot{y}_2^2)$
* V = $mgy_1 + mgy_2 + \frac{1}{2}kr_1^2 + \frac{1}{2}kr_2^2 $

In [ ]:
T = (1/2 * m1 * (smp.diff(x1, t)**2 + smp.diff(y1, t)**2 )) + (1/2 * m2 * (smp.diff(x2, t)**2 +  smp.diff(y2, t)**2))

V = m1*g*y1 + m2*g*y2 + 1/2 * k1 * r1**2 + 1/2 * k2 * r2**2

L = T-V

Compute Lagrange's equations

$\frac{dL}{dz} - \frac{d}{dt} \frac{dL}{d\dot{z}} = 0$

where $z$ is each of $\theta_1$, $r_1$, $\theta_2$, $r_2$

In [ ]:
LE1 = smp.diff(L, t1) - smp.diff(smp.diff(L, w1), t)
LE1 = LE1.simplify()

In [ ]:
LE2 = smp.diff(L, t2) - smp.diff(smp.diff(L, w2), t)
LE2 = LE2.simplify()

In [ ]:
LE3 = smp.diff(L, r1) - smp.diff(smp.diff(L, v1), t)
LE3 = LE3.simplify()

In [ ]:
LE4 = smp.diff(L, r2) - smp.diff(smp.diff(L, v2), t)
LE4 = LE4.simplify()

In [ ]:
LE1

If we solve for $d^2 z / d t^2$ where $z$ is each of $\theta_1$, $r_1$, $\theta_2$, $r_2$ then we can get two equation for each free variable. Defining $v_z$ as $dz/dt$ we get

* $dz/dt = v_z$
* $dv_z/dt = \text{whatever we solved for}$

This turns our system of second order ODES into systems 1D differential equations.

**Example** $\frac{d^2 y}{dt^2} + 2\frac{dy}{dt} + y + 3 = 0$ (define $v = dy/dt$) gets turned into the system of 2 first order DE's (i) $dy/dt = v$ and  (ii) $dv/dt =  - 3 - y - 2v$

Specifically, define 

* $\omega_1 \equiv d\theta_1/dt$
* $\omega_2 \equiv d\theta_2/dt$
* $v_1 \equiv dr_1/dt$
* $v_2 \equiv dr_2/dt$

In [ ]:
ddts = smp.solve([LE1, LE2, LE3, LE4], (dw1dt, dw2dt, dv1dt, dv2dt), simplify=False, rational=False)

In [ ]:
ddts[dw1dt]

Create numpy functions that we can use with numerical methods

In [ ]:
w1_f = smp.lambdify(w1, w1)
dw1dt_f = smp.lambdify((m1, m2, k1, k2, g, t1, t2, r1, r2, w1, w2, v1, v2), ddts[dw1dt])

w2_f = smp.lambdify(w2, w2)
dw2dt_f = smp.lambdify((m1, m2, k1, k2,g, t1, t2, r1, r2, w1, w2, v1, v2), ddts[dw2dt])

v1_f = smp.lambdify(v1, v1)
dv1dt_f = smp.lambdify((m1, m2, k1, k2,g, t1, t2, r1, r2, w1, w2, v1, v2), ddts[dv1dt])

v2_f = smp.lambdify(v2, v2)
dv2dt_f = smp.lambdify((m1, m2, k1, k2,g, t1, t2, r1, r2, w1, w2, v1, v2), ddts[dv2dt])

Define our system of ODES where $S = (\theta_1, \omega_1, \theta_2, \omega_2, r_1, v_1, r_2, v_2)$

In [ ]:
def dSdt(S, t):
    t1, w1, t2, w2, r1, v1, r2, v2 = S
    return [
        w1_f(w1),
        dw1dt_f(m1, m2, k1, k2,g, t1, t2, r1, r2, w1, w2, v1, v2),

        w2_f(w2),
        dw2dt_f(m1, m2, k1, k2,g, t1, t2, r1, r2, w1, w2, v1, v2),

        v1_f(v1),
        dv1dt_f(m1, m2, k1, k2,g, t1, t2, r1, r2, w1, w2, v1, v2),

        v2_f(v2),
        dv2dt_f(m1, m2, k1, k2,g, t1, t2, r1, r2, w1, w2, v1, v2)
    ]

Initial Condition

In [ ]:
# y0_1 = (np.pi/4, np.pi/4, 0, 0, 0, 3/2*np.pi/4, 1, 2)

Define times, constants, solve ODE

In [ ]:
t = np.linspace(0, 20, 1000)
g = 9.81
m1 = 1
m2 = 2
k1 = 10
k2 = 15
system_1 = odeint(dSdt, y0=[np.pi/4,0,(3/2)*np.pi/2,0,0,10,0,5], t = t)

Plot $\theta_1$ as function of time

In [ ]:
plt.plot(system_1.T[0])

Get locations of $x_1$, $y_1$, $x_2$, $y_2$ given $\theta_1$, $r_1$, $\theta_2$, $r_2$

In [ ]:
def get_x1y1x2y2(t1, t2, r1, r2):
    return ((1+r1)*np.cos(t1),
            -(1+r1)*np.sin(t1),
            (1+r1)*np.cos(t1) + (1+r2)*np.cos(t2),
            -(1+r1)*np.sin(t1)-(1+r2)*np.sin(t2)
    )

Get $x$s and $y$s as function of time

In [ ]:
x1, y1, x2, y2 = get_x1y1x2y2(system_1.T[0], system_1.T[2], system_1.T[4], system_1.T[6])

Plot ys

In [ ]:
plt.plot(y1)

Make animation 

In [ ]:
def animate(i):
    ln1.set_data([0, x1[i]], [0, y1[i]])
    ln2.set_data([x1[i], x2[i]], [y1[i], y2[i]])
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.grid()
ln1, = plt.plot([], [], 'bo--', lw=k1/4, markersize=m1*4)
ln2, = plt.plot([], [], 'ro--', lw=k2/4, markersize=m2*4)
ax.set_ylim(-18, 8)
ax.set_xlim(-10, 10)
ani = animation.FuncAnimation(fig, animate, frames=1000, interval=50)
ani.save('Double_Springed_Pendulum.gif',writer='pillow',fps=50)